In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

from scipy import optimize
from scipy import interpolate
import os
import sys

# import pathlib
# from tkinter import Tk, filedialog
# import re


In [2]:
def get_halfcellCapacity(Sto_pos_min, Sto_pos_max, Sto_neg_max, Sto_neg_min):
    C_pos = 1+ (1-Sto_pos_max) + Sto_pos_min
    C_neg = 1+ (1-Sto_neg_max) + Sto_neg_min

    return C_pos, C_neg

def normalize_array(array):
    """
    Normalizes an array between 0 and 1.
    """
    return (array - np.min(array)) / (np.max(array) - np.min(array))

def preprocess_data(data):
    """
    Preprocesses the data by removing NaN values and normalizing the data.
    """
    valid_mask = ~np.isnan(data[0]) & ~np.isnan(data[1])
    cleaned_data = [data[0][valid_mask], data[1][valid_mask]]
    cleaned_data[1] = normalize_array(cleaned_data[1])
    
    
    return cleaned_data

def interpolate_data(data, x_new):
    """
    Interpolates the given data over the new x-values.
    """
    return np.interp(x_new, data[1], data[0])#,left=np.nan,right=np.nan)

def ensure_correct_orientation(OCV, OCPPos, OCPNeg):
    """
    Ensures that the orientation of the OCV and OCP data is correct.
    """
    if OCPPos[0][0] > OCPPos[0][-1]:
        OCPPos[0] = np.flip(OCPPos[0])
        # OCPPos[1] = np.flip(OCPPos[1])
    if OCPNeg[0][0] < OCPNeg[0][-1]:
        OCPNeg[0] = np.flip(OCPNeg[0])
        # OCPNeg[1] = np.flip(OCPNeg[1])
    if OCV[0][0] > OCV[0][-1]:
        OCV[0] = np.flip(OCV[0])
        # OCV[1] = np.flip(OCV[1])
        
    return OCV, OCPPos, OCPNeg


def get_DVA(Charge, Voltage):
    return (np.diff(Voltage)/np.diff(Charge))
            

def get_stoichiometry(OCV, OCPPos, OCPNeg,initial_guess = [0.1, 0.99, 0.99, 0.01], bounds = [(0, 0.2), (0.95, 1), (0.95, 1), (0, 0.05)], Plot = False):
    """
    Determines the stoichiometric data from half-cell and full-cell data.
    """
    OCV = preprocess_data(OCV)
    OCPPos = preprocess_data(OCPPos)
    OCPNeg = preprocess_data(OCPNeg)

    # Ensure correct orientation of the data
    OCV, OCPPos, OCPNeg = ensure_correct_orientation(OCV, OCPPos, OCPNeg)

    def objective(x, plot=False,returnPlotData = False):
        Sto_pos_min, Sto_pos_max, Sto_neg_max, Sto_neg_min = x
        
        SoC = np.linspace(0, 1, 1000)
        Soc_Sto_neg = np.linspace(Sto_neg_min, Sto_neg_max, 1000)
        Soc_Sto_pos = np.linspace(Sto_pos_min, Sto_pos_max, 1000)
        Soc_Sto_pos = np.linspace(Sto_pos_min, Sto_pos_max, 1000)

        
        OCV_int = interpolate_data(OCV, SoC)
        OCPPos_int = interpolate_data(OCPPos, Soc_Sto_pos)
        OCPNeg_int = interpolate_data(OCPNeg, Soc_Sto_neg)
        
        
        OCPPos_int_norm = np.interp(SoC, normalize_array(Soc_Sto_pos), OCPPos_int)#,left=np.nan,right=np.nan)
        OCPNeg_int_norm = np.interp(SoC, normalize_array(Soc_Sto_neg), OCPNeg_int)#,left=np.nan,right=np.nan)
        
        # OCPPos_int_norm = interpolate_data( normalize_array(OCPPos_int)
        # OCPNeg_int_norm = normalize_array(OCPNeg_int)
        
        
        C_pos, C_neg = get_halfcellCapacity(Sto_pos_min, Sto_pos_max, Sto_neg_max, Sto_neg_min)
            
        if plot:
            fig, ax = plt.subplots()
            # ax.plot(SoC, OCPPos_int_norm, label='Pos')
            # ax.plot(SoC, OCPNeg_int_norm, label='Neg')
            #
            ax.plot((OCPPos[1]*C_pos)-Sto_pos_min, OCPPos[0], label='Pos')
            ax.plot((OCPNeg[1]*C_neg)-Sto_neg_min, OCPNeg[0], label='Neg')
            ax.plot(SoC, OCPPos_int_norm - OCPNeg_int_norm, label='Simulated', linestyle = '--')
            ax.plot(SoC, OCV_int, label='Measured')
            
            
            
            # ax[1].plot(SoC[0:-1], get_DVA(SoC, OCPPos_int_norm), label='Pos')
            # ax[1].plot(SoC[0:-1], get_DVA(SoC, OCPNeg_int_norm), label='Neg')
            # ax[1].plot(SoC[0:-1], get_DVA(SoC, OCPPos_int_norm - OCPNeg_int_norm), label='Simulated')
            # ax[1].plot(SoC[0:-1], get_DVA(SoC, OCV_int), label='Measured')
            # ax[1].set_ylim(-2,2)
            
            
            ax.legend()
            # ax[1].legend()
        if returnPlotData:
            plot_data = [
            {'x':(OCPPos[1]*C_pos)-Sto_pos_min, 'y':OCPPos[0], 'label': 'Pos'},
            {'x':(OCPNeg[1]*C_neg)-Sto_neg_min, 'y':OCPNeg[0], 'label': 'Neg'},
            {'x':SoC, 'y': OCPPos_int_norm - OCPNeg_int_norm, 'label': 'Simulated'},
            {'x':SoC, 'y': OCV_int, 'label': 'Measured'}
            ]
            return plot_data
        
        error = np.sqrt(np.sum((OCV_int - (OCPPos_int_norm - OCPNeg_int_norm))**2)/len(OCV_int))
        # error = np.sum((get_DVA(SoC, OCV_int) - get_DVA(SoC, OCPPos_int_norm - OCPNeg_int_norm))**2)
        return error if not np.isnan(error) else 10000000

    # initial_guess = [0.1, 0.99, 0.99, 0.01]
    # bounds = [(0, 0.3), (0.95, 1), (0.95, 1), (0, 0.05)]
    
    result = optimize.minimize(objective, initial_guess, bounds=bounds, method= 'Nelder-Mead')
    
    Sto_pos_min, Sto_pos_max, Sto_neg_max, Sto_neg_min = result.x
    error = result.fun
    if Plot:
        objective(result.x, plot=True,returnPlotData = False)
    plot_data = objective(result.x, plot=False, returnPlotData = True)    
        
    return Sto_pos_min, Sto_pos_max, Sto_neg_max, Sto_neg_min, error, plot_data